<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone repo data into environment
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

# All code was last ran on an AMD Ryzen 7 5800X CPU. Printed time values are not representative of wherever you are currently viewing this.

In [76]:
import pandas as pd
import folium
from folium import Map, Marker, PolyLine, features, RegularPolygonMarker, DivIcon
from folium.plugins import PolyLineTextPath
import math
import networkx as nx
import time
import multiprocessing
import concurrent

In [ ]:
# 2.3.1

nodes = pd.read_csv('tol403-lokaverkefni/data/nodes.tsv', sep = "\t")
edges = pd.read_csv('tol403-lokaverkefni/data/edges.tsv', sep = "\t")

charging_station_nodes = {323346405, 87120378, 2374444198, 1345740157, 2351742223}

coords = {
    row['osmid']: (row['y'], row['x'])  # Folium notar (lat, lon)
    for _, row in nodes.iterrows()
}

# búum til graf með stefnu og þyngd fyrir edges
G = nx.DiGraph()
for _, row in edges.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['length'])

## Need to reverse in some cases because it's directed
G_rev = G.reverse()

In [29]:
# 2.3.2

def shortest_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.dijkstra_path_length(graph, source=node_id, target=charger_id, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

In [50]:
# 2.3.3

def process_node_wrapper(args):
    row, charger_nodes, graph, charger_only = args
    node_id = row['osmid']
    
    color_palette = [
        "orange",
        "green",
        "purple",
        "cadetblue",
        "darkred",
        "darkgreen",
        "darkblue",
        "pink",
        "black",
        "gold"
    ]
    
    if node_id in charger_nodes:
        charger_index = list(charger_nodes).index(node_id) % len(color_palette)
        color = color_palette[charger_index]
        popup_text = f"🔌 Hleðslustöð <br>Node ID: {node_id}"
        return {
            'location': [row['y'], row['x']],
            'color': color,
            'popup_text': popup_text,
            'is_charger': True
        }
    
    elif not charger_only:
        distance, closest = shortest_distance_to_charger(node_id, graph, charger_nodes)
        
        if closest is None:
            color = "gray"  # fallback color if no reachable charger
            popup_text = (f"🚗 Node ID: {node_id}<br>Primary: {row['primary']}<br>"
                          f"⚠️ Engin hleðslustöð fannst í nágrenninu.")
        else:
            charger_index = list(charger_nodes).index(closest) % len(color_palette)
            color = color_palette[charger_index]
            popup_text = (f"🚗 Node ID: {node_id}<br>Primary: {row['primary']}<br>"
                          f"Fjarlægð frá næstu hleðslustöð: {distance:.2f} meters<br>"
                          f"Næsta hleðslustöð: {closest}")
        
        return {
            'location': [row['y'], row['x']],
            'color': color,
            'popup_text': popup_text,
            'is_charger': False
        }

# keyrsla gæti tekið sirka 5-10 mín
def create_map(charger_nodes, map_name, chargers_only = False):
    center_lat = nodes['y'].mean()
    center_lon = nodes['x'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    start_time = time.time()

    row_args = [
        (row, charger_nodes, G, chargers_only)
        for _, row in nodes.iterrows()
    ]

    # ryzen 7 go brrrr
    with multiprocessing.Pool() as pool:
        marker_data_list = pool.map(process_node_wrapper, row_args)

    chargers = []
    others = []

    for md in marker_data_list:
        if md is None:
            continue
        if md['is_charger']:
            chargers.append(md)
        else:
            others.append(md)

    # First draw regular nodes
    for md in others:
        folium.CircleMarker(
            location=md['location'],
            radius=2,
            color=md['color'],
            fill=True,
            fill_color=md['color'],
            fill_opacity=0.8,
            popup=folium.Popup(md['popup_text'], max_width=250)
        ).add_to(m)

    # Then draw charging stations (bigger + on top)
    for md in chargers:
        folium.CircleMarker(
            location=md['location'],
            radius=8,
            color="black",
            weight=2,
            fill=True,
            fill_color=md['color'],
            fill_opacity=1.0,
            popup=folium.Popup(md['popup_text'], max_width=250)
        ).add_to(m)

    print(f"Keyrsla tók {int(time.time() - start_time)}s")

    for _, row in edges.iterrows():
        if row['u'] in coords and row['v'] in coords:
            start = coords[row['u']]
            end = coords[row['v']]
            PolyLine(
                locations=[start, end],
                color='gray',
                weight=1,
                opacity=0.5,
                popup=row.get('name', '')
            ).add_to(m)

    m.save(map_name)
    print(f"Kort geymt i skra: {map_name}")

create_map(charging_station_nodes, "kort_2_3_3.html")

Keyrsla tók 25s
Kort geymt i skra: kort_2_3_3.html


In [31]:
# 2.3.4

## it's in the code above

In [32]:
# 2.3.5

## Heuristic for A*
## Needlessly complicated on such a small scale
## Calculate the distance between two points on the surface of a sphere
## It's somewhat off up here since the Earth isn't a perfect sphere
def calc_spherical_distance(node_id, target_id):
    lat_source,lon_source = coords[node_id]
    lat_target,lon_target = coords[target_id]

    radius = 6373.0

    delta_lat = math.radians(lat_target) - math.radians(lat_source)
    delta_lon = math.radians(lon_target) - math.radians(lon_source)

    a = math.sin(delta_lat / 2)**2 + math.cos(lat_source) * math.cos(lat_target) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c * 1000 #metres

    return distance

def astar_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.astar_path_length(graph, source=node_id, target=charger_id, heuristic=calc_spherical_distance, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

def process_astar_row(args):
    row, charging_station_nodes, graph = args
    node_id = row['osmid']

    distance, closest = astar_distance_to_charger(node_id, graph, charging_station_nodes)
    return distance if distance != float("inf") else 0.0

def run_astar(charging_station_nodes, benchmark):
    distance_sum = 0.0
    if benchmark:
        start_time = time.time()

    row_args = [(row, charging_station_nodes, G) for _, row in nodes.iterrows()]
    
    with multiprocessing.Pool() as pool:
        distances = pool.map(process_astar_row, row_args)

        distance_sum = sum(distances)

    if benchmark:
        print(f"Keyrsla tók {int(time.time() - start_time)}s")

    return distance_sum

run_astar(charging_station_nodes, True)

Keyrsla tók 19s


52496231.653249905

In [66]:
# 2.3.6

def simple_dijkstra(charger_id):
    distances = nx.single_source_dijkstra_path_length(G_rev, charger_id, weight='weight')
    distance_sum = sum(distances.values())

    return distance_sum

def multi_dijkstra(candidate_charger_id: int, placed_charger_ids: set):
    chargers = placed_charger_ids.copy()
    if candidate_charger_id is not None:
        chargers.add(candidate_charger_id)

    distances = nx.multi_source_dijkstra_path_length(G_rev, chargers, weight="weight")
    distance_sum = sum(distances.values())

    return distance_sum

def compute_distance(node, used_nodes):
    distance = multi_dijkstra(node.osmid, used_nodes)
    return node, distance

def place_optimal_charger(candidates, used_nodes):
    best_distance = float("inf")
    best_node = None

    with multiprocessing.Pool() as pool:
        results = pool.starmap(compute_distance, [(node, used_nodes) for node in candidates])

    for node, distance in results:
        if 0 < distance < best_distance:
            best_distance = distance
            best_node = node

    return best_node, best_distance

primary_nodes = []
for _, row in nodes.iterrows():
    if row['primary'] == True:
        primary_nodes.append(row)

optimal_node, distance = place_optimal_charger(primary_nodes[:], set())
print(f"Best node for k = 1 was {optimal_node} with a distance of {distance}")

Best node for k = 1 was osmid       34827739
x         -21.845736
y          64.114075
primary         True
Name: 589, dtype: object with a distance of 71813532.58901912


In [51]:
# 2.3.7

## Always add the new best node. Obviously have to remove the previously best node
def greedy_k_chargers(candidates, max_iterations, used_nodes):
    best_nodes = []
    total_distance = 0.0

    # If we already used some nodes, don't include them here
    for n in used_nodes:
        candidates[:] = [node for node in candidates if not node.equals(n)]

    start_time = time.time()
    for k in range(max_iterations):
        best_node, dist = place_optimal_charger(candidates, used_nodes)
        total_distance = dist
        best_nodes.append(best_node)
        used_nodes.add(best_node.osmid)
        # Python is awful btw
        candidates[:] = [node for node in candidates if not node.equals(best_node)]

    print(f"Keyrsla tók {int(time.time() - start_time)}s")
    return best_nodes, total_distance

greedy_nodes, greedy_dist = greedy_k_chargers(primary_nodes[:], 10, set())
print(f"Cumulative distance was {greedy_dist}")
create_map([node.osmid for node in greedy_nodes], "kort_2_3_7.html")

Keyrsla tók 28s
Cumulative distance was 24909868.79067309
Keyrsla tók 80s
Kort geymt i skra: kort_2_3_7.html


In [ ]:
# 2.3.8

## Pick random starting node
### Run greedy K for 5 nodes
#### Pick best node, then choose the furthest of the remaining 4 as the second best
##### Recurse
###### Pick solution with smallest distance at the end

def better_greedy_k(max_depth):
    # Choose a random starting node
    starting_node = int(nodes.sample(n=1).iloc[0]['osmid'])
    print(f"Starting from random node {starting_node}")
    with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
        chain, total_distance = recursive_search({starting_node}, max_depth, executor, 4)
    return chain, total_distance

def recursive_search(used_nodes: set, depth: int, executor=None, parallel_depth=0):
    if depth == 0:
        return used_nodes, multi_dijkstra(None, used_nodes)

    best_candidate, second_candidate = pick_2(used_nodes)

    best_used_nodes = used_nodes.copy()
    best_used_nodes.add(best_candidate.osmid)

    second_best_used_nodes = used_nodes.copy()
    second_best_used_nodes.add(second_candidate.osmid)

    if executor is not None and parallel_depth > 0:
        future1 = executor.submit(recursive_search, best_used_nodes, depth - 1, None, 0)
        future2 = executor.submit(recursive_search, second_best_used_nodes, depth - 1, None, 0)

        result1 = future1.result()
        result2 = future2.result()
    else:
        result1 = recursive_search(best_used_nodes, depth - 1, executor, parallel_depth - 1)
        result2 = recursive_search(second_best_used_nodes, depth - 1, executor, parallel_depth - 1)

    return result1 if result1[1] < result2[1] else result2
    
def pick_2(used_nodes):
    # Pick top 5 nodes
    chargers = top_5_candidates(primary_nodes[:], used_nodes)
    # First charger
    best_charger = chargers.pop(0)
    second_best = None

    max_distance = 0.0

    # Pick charger futhest from the best
    for k in chargers:
        dist = calc_spherical_distance(k.osmid, best_charger.osmid)
        if dist > max_distance:
            max_distance = dist
            second_best = k

    return best_charger, second_best

def top_5_candidates(candidates, used_nodes):
    best_nodes = []
    # If we already used some nodes, don't include them here
    for n in used_nodes:
        candidates[:] = [node for node in candidates if not node.equals(n)]

    with multiprocessing.Pool() as pool:
        results = pool.starmap(compute_distance_loc, [(node, used_nodes) for node in candidates])

    results.sort(key=lambda x: x[1])
    top_five = results[:5]

    best_nodes.extend([node for node, _ in top_five])

    return best_nodes

def compute_distance_loc(node, used_nodes):
    distance = multi_dijkstra(node.osmid, used_nodes)
    return node, distance

def get_best_results():
    best_distance = float("inf")
    best_nodes = []

    start_time = time.time()

    for _ in range(5):
        # 10th node is picked randomly at the start
        better_greedy_nodes, better_greedy_dist = better_greedy_k(9)
        if better_greedy_dist < best_distance:
            print(f"New best result of {better_greedy_dist}")
            best_distance = better_greedy_dist
            best_nodes = better_greedy_nodes

    end_time = time.time() - start_time
    print(f"Keyrsla tók {int(end_time)}s, avg {int(end_time / 5)}")

    return best_nodes, best_distance

best_greedy_nodes, best_greedy_dist = get_best_results()
print(f"Cumulative distance was {best_greedy_dist}")

Starting from random node 253472525
Cumulative distance was 24089418.20030137


In [81]:
print(f"Greedy algorithm picked {', '.join(str(node.osmid) for node in greedy_nodes)}, with a cumulative distance of {greedy_dist}")
print(f"Improved algorithm picked {best_greedy_nodes}, with a cumulative distance of {best_greedy_dist}")
create_map(best_greedy_nodes, "kort_2_3_8.html", False)

Greedy algorithm picked 34827739, 4159611763, 470316424, 1204996745, 470320635, 253702373, 2948755314, 62975525, 252165232, 251765347, with a cumulative distance of 24909868.79067309
Improved algorithm picked {661505312, 1208295841, 6382150947, 269072778, 14581771, 253472525, 14772879, 257168753, 2948755314, 2320771871}, with a cumulative distance of 24089418.20030137
Keyrsla tók 95s
Kort geymt i skra: kort_2_3_8.html
